# Importing the necessary libraries

In [ ]:

import pandas as pd 
from parsel import Selector
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException  # This line is important
import time
import random
import os
import re

opts = Options ()

# This section is for certificates

In [ ]:

# This function used for waiting
def wait_for_element_to_load(by=By.CLASS_NAME, name="pv-top-card", base=None):
    base = base or driver
    return WebDriverWait(base,5).until(
            EC.presence_of_element_located(
                (
                    by,
                    name
                )
            )
        )


def skill_parser(text):
    text = text.replace('·',',')
    text = text[7:].strip().split(",")
    for i in range(len(text)):
        text[i] = text[i].strip()
    result = ':'.join(text)
    if result == None:
        return "nan"
    return result


def certificationSection(url):
    try:
        url = os.path.join(url, "details/certifications/")
        driver.get(url)
        time.sleep(random.uniform(45.7, 50.3))

        main = wait_for_element_to_load(by=By.TAG_NAME, name="main")
        main_list = wait_for_element_to_load(name="pvs-list", base=main)
        certficate_name = ""
        certificate_issue_authority = ""
        certificate_issue_date = ""
        certificate_skill = ""
        for certification in main_list.find_elements(By.XPATH,"li"):
            certification_details = certification.find_element(By.CLASS_NAME,"pvs-entity")
            certification_details_list = certification_details.find_elements(By.XPATH,"*")
            certification_institution_elem = certification_details_list[1] if len(certification_details_list) > 1 else None
            lines = certification_institution_elem.text.split('\n')
            temp = []
            prev = ""
            for line in lines:
                if prev == line:
                    pass
                else:
                    temp.append(line)
                prev = line
            certficate_name = certficate_name + "-:-" + temp[0]
            certificate_issue_authority = certificate_issue_authority + "-:-" + temp[1] 
            certificate_issue_date = certificate_issue_date + "-:-" + temp[2]
        #             certficate_name.append(temp[0])
        #             certificate_issue_authority.append(temp[1])
        #             certificate_issue_date.append(temp[2])
            if len(temp)>3 and "credential" not in temp[3].lower():
                skill = temp[3]

                certificate_skill = certificate_skill + "-:-" + skill_parser(skill)
        #                 certificate_skill.append(skill_parser(skill))
            else:
                certificate_skill = certificate_skill + "-:-" + "nan"
                #certificate_skill.append("nan")
        return [certficate_name,certificate_issue_authority,certificate_issue_date,certificate_skill]
    except Exception as e:
        print(e)
        return ["nan","nan","nan", "nan"]
        

# This is solely for experience section


In [ ]:



def emptyChecker(string):
    if string:
        return string
    else:
        return "nan"

def wait_for_element_to_load(by=By.CLASS_NAME, name="pv-top-card", base=None):
    base = base or driver
    return WebDriverWait(base,5).until(
            EC.presence_of_element_located(
                (
                    by,
                    name
                )
            )
        )

def removingExtraLine(descrip):
    
    # Define a regular expression pattern to match a date range format
    date_pattern = r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4} - (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4} · \d+ mos\b"

    # Use re.search() to find the date range pattern in the first line
    match = re.search(date_pattern, descrip)

    # If a match is found in the first line, remove that line
    if match and match.start() == 0:
        lines = descrip.split('\n', 1)  # Split the data into lines, starting from the first newline
        if len(lines) > 1:
            descrip = lines[1]  # Keep the data after the first line
            
    return descrip


# Function to extract description
def extract_description(data):
    # Split the data by lines
    lines = data.strip().split('\n')
    
    # Extract the description part (after date)
    description = '\n'.join(lines[4:])
    description = removingExtraLine(description)
    return description


# url = "https://www.linkedin.com/in/apratim-chandra-singh-39a67a64/"

#The below function returns:
#[experience_section_position_title, experience_section_from_date, experience_section_to_date,experience_section_description , experience_section_duration, experience_section_company, experience_section_location]
#
def experienceSection(url):
    try: 
        url = os.path.join(url, "details/experience/")
        driver.get(url)
        time.sleep(random.uniform(55.7, 60.3))
        main = wait_for_element_to_load(by=By.TAG_NAME, name="main")
        main_list = wait_for_element_to_load(name="pvs-list", base=main)

        experience_section_position_title = ""
        experience_section_from_date = ""
        experience_section_to_date = ""
        experience_section_duration = ""
        experience_section_description = ""
        experience_section_company = ""
        experience_section_location = ""

        for position in main_list.find_elements(By.XPATH,"li"):
            position = position.find_element(By.CLASS_NAME,"pvs-entity")
            company_logo_elem, position_details = position.find_elements(By.XPATH,"*")

            # company elem
            company_linkedin_url = company_logo_elem.find_element(By.XPATH,"*").get_attribute("href")

            # position details
            position_details_list = position_details.find_elements(By.XPATH,"*")
            position_summary_details = position_details_list[0] if len(position_details_list) > 0 else None
            position_summary_text = position_details_list[1] if len(position_details_list) > 1 else None
            outer_positions = position_summary_details.find_element(By.XPATH,"*").find_elements(By.XPATH,"*")


            if len(outer_positions) == 4:
                position_title = outer_positions[0].find_element(By.TAG_NAME,"span").text
                company = outer_positions[1].find_element(By.TAG_NAME,"span").text
                work_times = outer_positions[2].find_element(By.TAG_NAME,"span").text
                location = outer_positions[3].find_element(By.TAG_NAME,"span").text
            elif len(outer_positions) == 3:
                if "·" in outer_positions[2].text:
                    position_title = outer_positions[0].find_element(By.TAG_NAME,"span").text
                    company = outer_positions[1].find_element(By.TAG_NAME,"span").text
                    work_times = outer_positions[2].find_element(By.TAG_NAME,"span").text
                    location = ""
                else:
                    position_title = ""
                    company = outer_positions[0].find_element(By.TAG_NAME,"span").text
                    work_times = outer_positions[1].find_element(By.TAG_NAME,"span").text
                    location = outer_positions[2].find_element(By.TAG_NAME,"span").text
            
            try:
                print("work_times: ", work_times)
            except:
                work_times = "nan"
                print("can't print work_T")
            times = work_times.split("·")[0].strip() if work_times else ""
            duration = work_times.split("·")[1].strip() if len(work_times.split("·")) > 1 else None

            from_date = " ".join(times.split(" ")[:2]) if times else ""
            to_date = " ".join(times.split(" ")[3:]) if times else ""

            if position_summary_text and len(position_summary_text.find_element(By.CLASS_NAME,"pvs-list").find_element(By.CLASS_NAME,"pvs-list").find_elements(By.XPATH,"li")) > 1:
                descriptions = position_summary_text.find_element(By.CLASS_NAME,"pvs-list").find_element(By.CLASS_NAME,"pvs-list").find_elements(By.XPATH,"li")
                for description in descriptions:
                    res = description.find_element(By.TAG_NAME,"a").find_elements(By.XPATH,"*")
                    position_title_elem = res[0] if len(res) > 0 else None
                    work_times_elem = res[1] if len(res) > 1 else None
                    location_elem = res[2] if len(res) > 2 else None


                    location = location_elem.find_element(By.XPATH,"*").text if location_elem else None
                    position_title = position_title_elem.find_element(By.XPATH,"*").find_element(By.TAG_NAME,"*").text if position_title_elem else ""
                    work_times = work_times_elem.find_element(By.XPATH,"*").text if work_times_elem else ""
                    times = work_times.split("·")[0].strip() if work_times else ""
                    duration = work_times.split("·")[1].strip() if len(work_times.split("·")) > 1 else None
                    from_date = " ".join(times.split(" ")[:2]) if times else ""
                    to_date = " ".join(times.split(" ")[3:]) if times else ""


                    if description:
                        text = description.text
                        description = extract_description(text)

                    experience_section_position_title = experience_section_position_title + "-:-" + emptyChecker(position_title)
                    experience_section_from_date = experience_section_from_date + "-:-" + emptyChecker(from_date)
                    experience_section_to_date = experience_section_to_date + "-:-" + emptyChecker(to_date)
                    experience_section_duration = experience_section_duration + "-:-" + emptyChecker(duration)
                    experience_section_description = experience_section_description + "-:-" + emptyChecker(description)
                    experience_section_company = experience_section_company + "-:-" + emptyChecker(company)
                    experience_section_location = experience_section_location + "-:-" + emptyChecker(location)
                    
            else:
                description = position_summary_text.text if position_summary_text else ""

                experience_section_position_title = experience_section_position_title + "-:-" + emptyChecker(position_title)
                experience_section_from_date = experience_section_from_date + "-:-" + emptyChecker(from_date)
                experience_section_to_date = experience_section_to_date + "-:-" + emptyChecker(to_date)
                experience_section_duration = experience_section_duration + "-:-" + emptyChecker(duration)
                experience_section_description = experience_section_description + "-:-" + emptyChecker(description)
                experience_section_company = experience_section_company + "-:-" + emptyChecker(company)
                experience_section_location = experience_section_location + "-:-" + emptyChecker(location)

        return [experience_section_position_title, experience_section_from_date, experience_section_to_date,experience_section_description , experience_section_duration, experience_section_company, experience_section_location]
    except Exception as e:
        print(e)
        return ["nan","nan", "nan", "nan", "nan",  "nan", "nan"]




# This is solely for educational feature

In [ ]:

def wait_for_element_to_load(by=By.CLASS_NAME, name="pv-top-card", base=None):
    base = base or driver
    return WebDriverWait(base,5).until(
            EC.presence_of_element_located(
                (
                    by,
                    name
                )
            )
        )

def emptyChecker(string):
    if string:
        return string
    else:
        return "nan"

# The function returns education_from_date,education_to_date,education_description,education_degree, 
# education_institution_name]
def educationInfo(url):
    try:
        url = os.path.join(url,"details/education/")
        driver.get(url)
        time.sleep(random.uniform(50.7, 55.3))
        
        main = wait_for_element_to_load(by = By.TAG_NAME, name = "main")
        main_list = wait_for_element_to_load(name="pvs-list", base=main)
        education_from_date = ""
        education_to_date = ""
        education_description = ""
        education_degree = ""
        education_institution_name = ""
        for position in main_list.find_elements(By.CLASS_NAME,"pvs-entity"):
                institution_logo_elem, position_details = position.find_elements(By.XPATH,"*")
                # company elem
                institution_linkedin_url = institution_logo_elem.find_element(By.XPATH,"*").get_attribute("href")

                # position details
                position_details_list = position_details.find_elements(By.XPATH,"*")
                position_summary_details = position_details_list[0] if len(position_details_list) > 0 else None
                position_summary_text = position_details_list[1] if len(position_details_list) > 1 else None
                outer_positions = position_summary_details.find_element(By.XPATH,"*").find_elements(By.XPATH,"*")

                institution_name = outer_positions[0].find_element(By.TAG_NAME,"span").text
                degree = outer_positions[1].find_element(By.TAG_NAME,"span").text

                if len(outer_positions) > 2:
                    times = outer_positions[2].find_element(By.TAG_NAME,"span").text
                    dates = times.split('-')
                    from_date = dates[0].strip()
                    to_date = dates[1].strip()
                else:
                    from_date = None
                    to_date = None

                description = position_summary_text.text if position_summary_text else ""
                print()
                education_from_date = education_from_date + "-:-" + emptyChecker(from_date)
                education_to_date = education_to_date + "-:-" + emptyChecker(to_date)
                education_description = education_description + "-:-" + emptyChecker(description)
                education_degree = education_degree + "-:-" + emptyChecker(degree)
                education_institution_name = education_institution_name + "-:-" + emptyChecker(institution_name)
                
#                 education_from_date.append(emptyChecker(from_date))
#                 education_to_date.append(emptyChecker(to_date))
#                 education_description.append(emptyChecker(description))
#                 education_degree.append(emptyChecker(degree))
#                 education_institution_name.append(emptyChecker(institution_name))

#                 print("From Date:", from_date)
#                 print("To Date:", to_date)
#                 print("Description:", description)
#                 print("Degree:", degree)
#                 print("Institution Name:", institution_name)
#                 print("********************")

        return [education_from_date,education_to_date,education_description,education_degree, education_institution_name]
            
    except Exception as e:
        print(e)
        
        return ["nan", "nan", "nan", "nan", "nan"]

# Main Code For Signing into Linkedin

In [ ]:
driver = webdriver.Chrome()

# Function to ensure all key data fields have a value

def validate_field(field):
    #if field is present pass if field:
    if field:
        pass
    #if field is not present print text else:
    else:
        field = 'No results'
    return field

#driver get method() will navigate to a page given by the URL address.
driver.get("https://www.linkedin.com/")

#locate email form by_class_name
username = driver.find_element(By.ID, 'session_key')

#send_keys() to simulate key strokes
username.send_keys('ENTER USERNAME')

#sleep for 0.5 seconds
sleep(0.5)

#Locate password form by_class_name
password = driver.find_element(By.ID, 'session_password')

#send_keys() to simulate key strokes
password.send_keys('ENTER PASSWORD')
sleep(0.5)

#Locate submit button by_xpath
sign_in_button = driver.find_element(By.XPATH,'//*[@type="submit"]')

#.click() to mimic button click
sign_in_button.click()

sleep(10)


In [ ]:
#sleep(15)

csai = ["Business Analyst"]


def role_tokeniser(job_role):
    job_role  = job_role.replace(" ","+")
    return job_role


Jobdata = []
lnks = []
role = csai[0]
for x in range(0,100,10):
    

    job_role = role_tokeniser(role)
    link = f'https://www.google.com/search?q=site:linkedin.com/in/+AND+%22{job_role}%22+AND+%22India%22&rlz=1C1CHZO_enIN1023IN1023&ei=jPaIY-mEGM6cseMPyZaNmAo&start={x}'
    print(link)
    driver.get(link)
    time.sleep(random.uniform(2.5,4.9))
    print("checkpoint1")
    try:
        linkedin_url = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//a[@jsname='UWckNb']"))).get_attribute("href")
        print("working:", linkedin_url)
        linkedin_urls = [my_elem.get_attribute("href") for my_elem in WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, "//a[@jsname='UWckNb']")))]
        print("checkpoint3")
        lnks.append(linkedin_urls)
        print("checkpoint4")
    except TimeoutException:
        print("TimeoutException: Elements not found within the specified time.")




# Start Scraping

In [ ]:

profile_data = []
flag = True
for x in lnks:
    newProfile = {}
    for i in x:
        i = i + "/"
        print("i", i)
        # Get the profile URL
        driver.get(i)
        time.sleep(random.uniform(2.5, 4.9))
        newProfile = {}

        # Assigning the source code for the web page to a variable
        page_source = driver.page_source
        sel = Selector(text=page_source)

        # Extract name
        name_element = driver.find_element(By.XPATH, '//h1[contains(@class, "text-heading-xlarge inline t-24 v-align-middle break-words")]')
        name = name_element.text.strip() if name_element else None
        print(name)
        
        # Extract job title
        try:
            job_title_element = driver.find_element(By.XPATH, '//div[contains(@class, "text-body-medium break-words")]')
        except:
            job_title_element = "nan"
        job_title = job_title_element.text.strip() if job_title_element else None
        print(job_title)
        
        #Extract about section
        try:
            about = driver.find_element(By.XPATH,'//div[contains(@class, "pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center")]').text
        except:
            about = "nan"
            
        #Extracting the education
        education = educationInfo(i)
        print(education)
        
        
#         #Extracting the experience
        # experience = experienceSection(i)
        # print(experience)
        
        
#         #Extracting the certificate
#         certification = certificationSection(i)
#         print(certification)
        
        #linkedin link
        link = i
        newProfile["name"] = [name]
        newProfile["currentJobTitle"] = job_title
        newProfile["about"] = about
        newProfile["education_from_date"] = education[0]
        newProfile["education_to_date"] = education[1]
        newProfile["education_description"] = education[2]
        newProfile["education_degree"] = education[3]
        newProfile["education_institution_name"] = education[4]
        # newProfile["experience_section_position_title"] = experience[0]
        # newProfile["experience_section_from_date"] = experience[1]
        # newProfile["experience_section_to_date"] = experience[2]
        # newProfile["experience_section_description"] = experience[3]
        # newProfile["experience_section_duration"] = experience[4]
        # newProfile["experience_section_company"] = experience[5]
        # newProfile["experience_section_location"] = experience[6]
        # newProfile["certficate_name"] = certification[0]
        # newProfile["certificate_issue_authority"] = certification[1]
        # newProfile["certificate_issue_date"] = certification[2]
        # newProfile["certificate_skill"] = certification[3]
        newProfile["linkedin_link"] = link
        if flag:
            df = pd.DataFrame(newProfile)
            flag = False
        df = df.append(newProfile, ignore_index=True)
        with pd.ExcelWriter("linkedin_profiles.xlsx", mode='a', engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='Sheet1', header=False)  # Set header=False to not overwrite existing headers

driver.quit()